<a href="https://colab.research.google.com/github/kyeltech/NLP_Assessment/blob/main/assessment_topic_modeling_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# loading data
!pip install pyLDAvis

In [2]:
# Importing modules
import pandas as pd
import os

ModuleNotFoundError: No module named 'pandas'

In [7]:
# uncomment below if you have the csv file in your directory
# %ls
news = pd.read_csv('bbc_news.csv')
# news = pd.read_csv('bbc_news.csv')


In [8]:
# Print head
news.head()


,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [9]:
# describe
# news.describe()
news.describe(include='all')
# End loading data

,title,pubDate,guid,link,description
count,32182,32182,32182,32182,32182
unique,30746,31308,29578,29341,30095
top,Ukraine war in maps: Tracking the Russian inva...,"Tue, 13 Jun 2023 10:25:22 GMT",https://www.bbc.co.uk/news/business-61634959,https://www.bbc.co.uk/news/business-61634959?a...,How closely have you been paying attention to ...
freq,32,8,63,61,80


In [10]:
# Start Data Cleaning
# Load the regular expression library
import re


news['description'].map(lambda x: re.sub('[,\.!?]', '', x)).map(lambda x: x.lower())


# Print out the first rows of papers
news['description'].head()

0    The Ukrainian president says the country will ...
1    Jeremy Bowen was on the frontline in Irpin, as...
2    One of the world's biggest fertiliser firms sa...
3    The parents of the Manchester Arena bombing's ...
4    Consumers are feeling the impact of higher ene...
Name: description, dtype: object

In [3]:
# Import the wordcloud library
from wordcloud import WordCloud

# Join the different processed titles together.
long_string = ','.join(list(news['description'].values))

# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()

NameError: name 'news' is not defined

In [2]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use' , 'said' , 'like' , 'us' , 'go','one', 'two', 'three', 'see'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]


data = news.description.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'news' is not defined

In [22]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]


In [20]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.018*"says" + 0.008*"new" + 0.008*"government" + 0.008*"russia" + '
  '0.007*"ukraine" + 0.007*"president" + 0.005*"six" + 0.005*"shows" + '
  '0.005*"fatally" + 0.005*"house"'),
 (1,
  '0.018*"bbc" + 0.010*"says" + 0.006*"tells" + 0.006*"death" + '
  '0.006*"actress" + 0.005*"told" + 0.005*"say" + 0.005*"mall" + 0.005*"year" '
  '+ 0.004*"uk"'),
 (2,
  '0.014*"seven" + 0.014*"days" + 0.013*"going" + 0.008*"past" + 0.007*"says" '
  '+ 0.007*"taken" + 0.006*"alan" + 0.006*"stabbed" + 0.006*"world" + '
  '0.006*"day"'),
 (3,
  '0.018*"israel" + 0.012*"says" + 0.010*"attack" + 0.008*"year" + '
  '0.008*"israeli" + 0.008*"say" + 0.007*"attacked" + 0.006*"people" + '
  '0.006*"old" + 0.006*"legal"'),
 (4,
  '0.020*"people" + 0.013*"uk" + 0.010*"says" + 0.008*"rates" + 0.006*"bbc" + '
  '0.006*"millions" + 0.005*"new" + 0.005*"bank" + 0.005*"government" + '
  '0.005*"interest"'),
 (5,
  '0.022*"league" + 0.019*"manchester" + 0.016*"city" + 0.011*"premier" + '
  '0.009*"iran" + 0.009

In [23]:
import pyLDAvis.gensim_models
import pickle
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('/content_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'content_'+ str(num_topics) +'.html')

LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.113561 -0.145771       1        1  11.831479
2      0.114283 -0.082796       2        1  11.277711
4     -0.092032 -0.003003       3        1  11.208472
7     -0.035306  0.061084       4        1  10.528844
3     -0.127307  0.042728       5        1   9.871639
5      0.152088 -0.120645       6        1   9.419737
9      0.081492  0.039567       7        1   9.367702
6      0.019577  0.016208       8        1   9.361935
1     -0.118352  0.033311       9        1   8.578310
8      0.119118  0.159317      10        1   8.554170, topic_info=           Term         Freq        Total Category  logprob  loglift
581        park  1175.000000  1175.000000  Default  30.0000  30.0000
2994      purim  1094.000000  1094.000000  Default  29.0000  29.0000
33      parents   884.000000   884.000000  Default  28.0000  28.0000
486       cover   587.000000   587.000000  Default  27.0000  27.0000
1085        run   639.000000   639.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
58    oligarchs   118.901502  1386.231134  Topic10  -5.5132   0.0027
25        shock   109.689033  1051.275253  Topic10  -5.5939   0.1986
205         see   104.576998  1222.908412  Topic10  -5.6416  -0.0003
615        last   102.160614   991.706106  Topic10  -5.6650   0.1859
256     lynette   100.554617   634.465279  Topic10  -5.6808   0.6167

[743 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
6149       8  0.967663     aanoch
3918       8  0.973381        abe
2811       1  0.974269  aberfeldy
2811       2  0.010947  aberfeldy
3277       2  0.039430    academy
...      ...       ...        ...
130        7  0.009441       year
130        8  0.007081       year
130        9  0.021243       year
130       10  0.004721       year
10590      6  0.937922        zuu

[2403 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 8, 4, 6, 10, 7, 2, 9])